In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np

np.set_printoptions(suppress=True) # turn of scientific notation
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import load_iris
from sklearn import metrics

torch.__version__

'2.0.1'

In [2]:
torch.backends.mps.is_available()

True

In [3]:
device = torch.device('mps')
x = torch.ones(5, device=device)
x

tensor([1., 1., 1., 1., 1.], device='mps:0')

In [4]:
# matrix 1x2
mat_1 = torch.tensor([[3,3]], device=device)
print(mat_1)
print(mat_1.size())

tensor([[3, 3]], device='mps:0')
torch.Size([1, 2])


In [5]:
x = torch.tensor([1,2], device=device)
y = torch.tensor([2,3], device=device)
print(x)

tensor([1, 2], device='mps:0')


In [6]:
sub = torch.subtract(x,y)
print(sub)

tensor([-1, -1], device='mps:0')


### MPG dataset

In [12]:
#df = pd.read_csv('https://raw.githubusercontent.com/rishi-wqd190004/DS_ML_Lectures/main/datasets/auto-mpg.csv', na_values=["NA", "?"])
df = pd.read_csv('/Users/rishinigam/online_teaching_freelance/DS_ML_Lectures/datasets/auto-mpg.csv', na_values=["NA", "?"])
df.head(2)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    392 non-null    float64
 4   weight        398 non-null    int64  
 5   acceleration  398 non-null    float64
 6   model year    398 non-null    int64  
 7   origin        398 non-null    int64  
 8   car name      398 non-null    object 
dtypes: float64(4), int64(4), object(1)
memory usage: 28.1+ KB


In [14]:
cars = df['car name']
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].median())

In [15]:
df.isna().sum()

mpg             0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
model year      0
origin          0
car name        0
dtype: int64

In [16]:
df['horsepower'].isnull().sum()

0

In [17]:
x = df[['cylinders', 'displacement', 'horsepower', 'weight','acceleration', 'model year', 'origin']].values
x.shape

(398, 7)

In [18]:
x

array([[  8. , 307. , 130. , ...,  12. ,  70. ,   1. ],
       [  8. , 350. , 165. , ...,  11.5,  70. ,   1. ],
       [  8. , 318. , 150. , ...,  11. ,  70. ,   1. ],
       ...,
       [  4. , 135. ,  84. , ...,  11.6,  82. ,   1. ],
       [  4. , 120. ,  79. , ...,  18.6,  82. ,   1. ],
       [  4. , 119. ,  82. , ...,  19.4,  82. ,   1. ]])

In [19]:
print(type(x))

<class 'numpy.ndarray'>


In [20]:
y = df['mpg'].values

In [21]:
y.shape

(398,)

In [22]:
# converting to torch
# x_ = torch.from_numpy(x)
x = torch.tensor(x, device=device, dtype=torch.float32)
y = torch.tensor(y, device=device, dtype=torch.float32)

### Model

In [23]:
model = nn.Sequential(
    nn.Linear(x.shape[1], 50),
    nn.ReLU(),
    nn.Linear(50,25),
    nn.ReLU(),
    nn.Linear(25,1)
)

# compiling the model
model = torch.compile(model, backend='aot_eager').to(device)

# Loss function for model
loss_fn = nn.MSELoss()

# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [24]:
# train for 1000 epoch
for epoch in range(1000):
    optimizer.zero_grad()
    out = model(x).flatten()
    loss = loss_fn(out, y)
    loss.backward()
    optimizer.step()

    # display 100 epochs
    if epoch % 100 == 0:
        print('Epoch {}, Loss: {}'.format(epoch, loss.item()))

Epoch 0, Loss: 25764.30859375
Epoch 100, Loss: 92.79591369628906
Epoch 200, Loss: 49.707252502441406
Epoch 300, Loss: 30.580875396728516
Epoch 400, Loss: 17.49102210998535
Epoch 500, Loss: 13.057997703552246
Epoch 600, Loss: 13.359935760498047
Epoch 700, Loss: 12.263630867004395
Epoch 800, Loss: 12.11543083190918
Epoch 900, Loss: 12.032702445983887


In [25]:
pred = model(x)
print(pred.shape)
print(pred[:10])

torch.Size([398, 1])
tensor([[16.0855],
        [14.8579],
        [16.3408],
        [16.6450],
        [16.0679],
        [10.3130],
        [10.0436],
        [10.1557],
        [ 9.8828],
        [13.0855]], device='mps:0', grad_fn=<SliceBackward0>)


- bringing back predictions from cpu 

In [26]:
score = np.sqrt(metrics.mean_squared_error(pred.cpu().detach(), y.cpu().detach()))
print('Fianl RMSE: {}'.format(score))

Fianl RMSE: 3.4922633171081543


In [27]:
# scoring using torch
score = torch.sqrt(nn.functional.mse_loss(pred.flatten(), y))
print('Fianl RMSE: {}'.format(score))

Fianl RMSE: 3.492263078689575


### Iris data classification

In [28]:
# loading iris from sklearn.datasets
x, y = load_iris(as_frame=True, return_X_y=True)
x.shape, y.shape

((150, 4), (150,))

In [29]:
y

0      0
1      0
2      0
3      0
4      0
      ..
145    2
146    2
147    2
148    2
149    2
Name: target, Length: 150, dtype: int64

In [30]:
tar_name = load_iris()['target_names']
tar_name

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [31]:
x.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [32]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: target, dtype: int64

In [33]:
cols = x.columns

In [34]:
y.value_counts()

target
0    50
1    50
2    50
Name: count, dtype: int64

In [35]:
# label encoding the target values
x = x.values
le = LabelEncoder()
y = le.fit_transform(y)
species = le.classes_

In [36]:
# converting to torch tensors and moving to mps:0 (metal GPU)
x = torch.tensor(x, device=device, dtype=torch.float32)
y = torch.tensor(y, device=device, dtype=torch.int64)

In [37]:
# modelling
model = nn.Sequential(
    nn.Linear(x.shape[1], 50),
    nn.ReLU(),
    nn.Linear(50,25),
    nn.ReLU(),
    nn.Linear(25, len(species)),
    nn.LogSoftmax(dim=1)
)

model = torch.compile(model, backend="aot_eager").to(device)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()
for epoch in range(1000):
    optimizer.zero_grad()
    out = model(x)
    loss = criterion(out, y)
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print("Epoch {}, loss {}".format(epoch, loss.item()))

Epoch 0, loss 1.0848673582077026
Epoch 100, loss 0.05183012783527374
Epoch 200, loss 0.04421135038137436
Epoch 300, loss 0.047979190945625305
Epoch 400, loss 0.04037211835384369
Epoch 500, loss 0.03940005600452423
Epoch 600, loss 0.03948890045285225
Epoch 700, loss 0.039229750633239746
Epoch 800, loss 0.039258923381567
Epoch 900, loss 0.03849180042743683


In [38]:
print(species)

[0 1 2]


In [39]:
# evaluating model
model.eval()
pred = model(x)
print('Shape of prediction: {}'.format(pred.shape))
print(pred[:10])

Shape of prediction: torch.Size([150, 3])
tensor([[-2.0266e-06, -1.3082e+01, -2.9913e+01],
        [-1.3232e-05, -1.1231e+01, -2.7946e+01],
        [-5.3644e-06, -1.2143e+01, -2.8376e+01],
        [-1.7404e-05, -1.0957e+01, -2.7348e+01],
        [-1.6689e-06, -1.3287e+01, -2.9962e+01],
        [-2.0266e-06, -1.3092e+01, -3.0122e+01],
        [-5.4836e-06, -1.2108e+01, -2.8040e+01],
        [-4.5299e-06, -1.2313e+01, -2.9158e+01],
        [-2.7060e-05, -1.0518e+01, -2.6558e+01],
        [-1.1325e-05, -1.1388e+01, -2.8445e+01]], device='mps:0',
       grad_fn=<SliceBackward0>)


In [40]:
# moving to cpu to print float values
print(pred[:10].cpu().detach().numpy())

[[ -0.00000203 -13.081657   -29.912601  ]
 [ -0.00001323 -11.2307005  -27.946493  ]
 [ -0.00000536 -12.143249   -28.376152  ]
 [ -0.0000174  -10.957186   -27.347845  ]
 [ -0.00000167 -13.287111   -29.962229  ]
 [ -0.00000203 -13.092123   -30.122362  ]
 [ -0.00000548 -12.108203   -28.040232  ]
 [ -0.00000453 -12.312765   -29.15807   ]
 [ -0.00002706 -10.51829    -26.557777  ]
 [ -0.00001132 -11.387964   -28.444893  ]]


In [41]:
_, predicted_class = torch.max(pred,1)
print('Expected: ',predicted_class)
print('Actual: ', y)

Expected:  tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2], device='mps:0')
Actual:  tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2

In [43]:
# accuracy
acc = metrics.accuracy_score(y.cpu().detach(), predicted_class.cpu().detach())
print('Accuracy: ', acc)

Accuracy:  0.9866666666666667


In [42]:
# metrics for evaluation
score = np.sqrt(metrics.mean_squared_error(pred.cpu().detach(), y.cpu().detach()))
print('RMSE score: {}'.format(score))

ValueError: y_true and y_pred have different number of output (3!=1)